In [1]:
from config import *

In [2]:
# tf.dataset return data in X, y format
resize_rescale = tf.keras.models.Sequential([
            tf.keras.layers.Resizing(224, 224, crop_to_aspect_ratio=True),
            tf.keras.layers.Rescaling(1./255)
])

# Data augment layer
data_augment = tf.keras.models.Sequential([
            tf.keras.layers.RandomFlip("horizontal_and_vertical"),
            tf.keras.layers.RandomRotation(0.2),
            tf.keras.layers.RandomContrast(factor=0.2)

]) 

def preprocess(df, shuffle=False, augment=False):
    tdf = df.map(lambda X,y: (resize_rescale(X), y))
    if shuffle:
        tdf = tdf.shuffle(1_000,seed=42)
    # Batch all dataset
    tdf = tdf.batch(18)
    #Augmentation on training set: training=True means it will on apply during training
    if augment:
        tdf = tdf.map(lambda X,y: (data_augment(X, training=True), y))
    return tdf.prefetch(1)

train_data = preprocess(train_set_raw,True,True)
test_data  = preprocess(test_set_raw)
valid_data = preprocess(valid_set_raw)

In [6]:
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self, n_neurons1,n_neurons2,n_neurons3,n_neurons4,n_neurons5,n_neurons6, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = tf.keras.layers.Conv2D(filters=n_neurons1,kernel_size=1,strides=1,padding='same',activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(filters=n_neurons2,kernel_size=3,strides=1,padding='same',activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(filters=n_neurons3,kernel_size=5,strides=1,padding='same',activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(filters=n_neurons4,kernel_size=1,strides=1,padding='same',activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(filters=n_neurons5,kernel_size=1,strides=1,padding='same',activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(filters=n_neurons6,kernel_size=1,strides=1,padding='same',activation='relu')
        self.max   = tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=1,padding='same')
        self.concat= tf.concat

    def call(self, inputs):
        Z = inputs
        conv1 = self.conv1(Z)
        conv2 = self.conv2(self.conv5(Z))
        conv3 = self.conv3(self.conv6(Z))
        conv4 = self.conv4(self.max(Z))
        return self.concat([conv1,conv2,conv3,conv4], axis=3)

In [26]:
cls()
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64,kernel_size=7,strides=2,activation='relu',padding='same',input_shape=[224,224,3]),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=3,strides=1,activation='relu',padding='same'),
    tf.keras.layers.Conv2D(filters=192,kernel_size=3,strides=1,activation='relu',padding='same'),
    tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x)),
    tf.keras.layers.MaxPool2D(),
    InceptionModule(64,128,32,32,96,16),
    InceptionModule(128,192,96,64,128,32),
    tf.keras.layers.MaxPool2D(),
    InceptionModule(192,208,48,64,96,16),
    InceptionModule(160,224,64,64,112,24),
    InceptionModule(128,256,64,64,128,24),
    InceptionModule(112,288,64,64,144,32),
    InceptionModule(256,320,128,128,160,32),
    tf.keras.layers.MaxPool2D(),
    InceptionModule(256,320,128,128,160,32),
    InceptionModule(384,384,128,128,192,48),
    tf.keras.layers.GlobalAveragePooling2D(keepdims=True),
    tf.keras.layers.Dropout(.4),
    tf.keras.layers.Dense(1000,activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 64)      9472      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
lambda (Lambda)              (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 192)       110784    
_________________________________________________________________
lambda_1 (Lambda)            (None, 56, 56, 192)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 192)       0

In [27]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])
hist = model.fit(train_data,validation_data=(valid_data), epochs=1)
score = model.evaluate(test_data)
score

21/21 [==============================] - 10s 480ms/step - loss: 1.6052 - accuracy: 0.2420


[1.60515296459198, 0.24199724197387695]